# Chess AI Prediction against Stockfish

This notebook demonstrates how to use the trained model to play against Stockfish or other chess engines.

## Setup and Imports

In [2]:
%pip install stockfish python-chess torch numpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import numpy as np
import pickle
from chess import Board, Move
from aux_func import board_to_matrix
from model import ChessModel
from stockfish import Stockfish

## Load Model and Move Dictionary

In [4]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load move encoding dictionary
with open('./models/move_to_int.pkl', 'rb') as f:
    move_to_int = pickle.load(f)

int_to_move = {v: k for k, v in move_to_int.items()}
num_classes = len(move_to_int)

print(f"Number of possible moves in training data: {num_classes}")
print(f"Device: {device}")

Number of possible moves in training data: 1892
Device: cpu


In [5]:
# Initialize and load model
model = ChessModel(num_classes).to(device)
model.load_state_dict(torch.load('./models/10Epoch_chessModel.pth', map_location=device))
model.eval()
print("Model loaded successfully!")

Model loaded successfully!


## Helper Functions

In [ ]:
from IPython.display import display

def get_model_move(board):
    """
    Get the model's best move for the current board position.
    Returns the highest confidence legal move.
    """
    with torch.no_grad():
        # Convert board to matrix
        board_matrix = board_to_matrix(board)
        board_tensor = torch.tensor(board_matrix, dtype=torch.float32).unsqueeze(0).to(device)
        
        # Get model predictions
        output = model(board_tensor)
        probabilities = torch.softmax(output, dim=1)[0]
        
        # Get legal moves
        legal_moves = list(board.legal_moves)
        legal_moves_uci = [move.uci() for move in legal_moves]
        
        # Find highest confidence legal move
        best_move = None
        best_score = -1
        
        for move_uci in legal_moves_uci:
            if move_uci in move_to_int:
                move_idx = move_to_int[move_uci]
                score = probabilities[move_idx].item()
                if score > best_score:
                    best_score = score
                    best_move = Move.from_uci(move_uci)
        
        # Fallback to random legal move if no move found in training data
        if best_move is None:
            import random
            best_move = random.choice(legal_moves)
            best_score = 0.0
        
        return best_move, best_score

def play_game(model_is_white=True, stockfish_path=None, max_moves=100, d=10):
    """
    Play a game between the model and Stockfish.
    Args:
        model_is_white: If True, model plays as white; else as black
        stockfish_path: Path to stockfish executable (auto-detect if None)
        max_moves: Maximum number of moves before draw
    Returns:
        Game result and move history
    """
    board = Board()
    
    # Initialize Stockfish
    try:
        if stockfish_path:
            sf = Stockfish(path=stockfish_path, depth=d)
        else:
            sf = Stockfish(depth=d)
    except Exception as e:
        print(f"Error initializing Stockfish: {e}")
        print("Please install stockfish: https://stockfishchess.org/download/")
        return None
    
    move_history = []
    move_count = 0
    
    print(f"\nGame Started - Model: {'White' if model_is_white else 'Black'}, Stockfish: {'Black' if model_is_white else 'White'}")
    print("="*60)
    
    while not board.is_game_over() and move_count < max_moves:
        is_white_turn = board.turn
        
        if is_white_turn == model_is_white:
            # Model's turn
            move, confidence = get_model_move(board)
            print(f"Model ({['Black', 'White'][model_is_white]}): {move.uci()} (confidence: {confidence:.4f})")
        else:
            # Stockfish's turn
            sf.set_fen_position(board.fen())
            best_move_uci = sf.get_best_move()
            if best_move_uci:
                move = Move.from_uci(best_move_uci)
                print(f"Stockfish ({['Black', 'White'][not model_is_white]}): {move.uci()}")
            else:
                print("Stockfish: No move available (checkmate/stalemate)")
                break
        
        board.push(move)
        move_history.append(move.uci())
        move_count += 1
        display(board)
    
    # Print result
    print("="*60)
    print(f"Game Over - Total moves: {move_count}")
    
    if board.is_checkmate():
        winner = "Stockfish" if board.turn == model_is_white else "Model"
        print(f"Result: Checkmate - {winner} wins!")
    elif board.is_stalemate():
        print("Result: Stalemate - Draw")
    elif board.is_insufficient_material():
        print("Result: Insufficient material - Draw")
    elif move_count >= max_moves:
        print(f"Result: Draw (reached {max_moves} moves)")
    else:
        print(f"Result: Game ended - {board.outcome()}")
    
    return move_history

def analyze_position(fen=None):
    """
    Analyze a specific chess position.
    """
    if fen is None:
        board = Board()
    else:
        board = Board(fen)
    
    move, confidence = get_model_move(board)
    print(f"Model's recommended move: {move.uci()}")
    print(f"Confidence: {confidence:.4f}")
    display(board)
    
    return move

## Play Against Stockfish

In [ ]:
# Play a game - Model as White
# Note: You must have Stockfish installed. Download from https://stockfishchess.org/download/
# On Windows, specify the path: stockfish_path=r'C:\path\to\stockfish.exe'

game_history = play_game(model_is_white=True, stockfish_path=r"./stockfish/stockfish-windows-x86-64-avx2.exe", max_moves=200, d=5)


Game Started - Model: White, Stockfish: Black
Model (White): e2e4 (confidence: 0.4121)
Stockfish (Black): e7e5
Model (White): g1f3 (confidence: 0.7764)
Stockfish (Black): b8c6
Model (White): f1b5 (confidence: 0.7078)
Stockfish (Black): a7a6
Model (White): b5a4 (confidence: 0.9494)
Stockfish (Black): g8f6
Model (White): e1g1 (confidence: 0.6781)
Stockfish (Black): f6e4
Model (White): d2d4 (confidence: 0.9879)
Stockfish (Black): b7b5
Model (White): a4b3 (confidence: 0.9992)
Stockfish (Black): d7d5
Model (White): d4e5 (confidence: 0.8351)
Stockfish (Black): c8e6
Model (White): b1d2 (confidence: 0.2689)
Stockfish (Black): e4c5
Model (White): c2c3 (confidence: 0.2424)
Stockfish (Black): e6f5
Model (White): b3c2 (confidence: 0.2561)
Stockfish (Black): f5c2
Model (White): d1e1 (confidence: 0.3939)
Stockfish (Black): c5e6
Model (White): d2b3 (confidence: 0.4590)
Stockfish (Black): f8e7
Model (White): b3d4 (confidence: 0.2135)
Stockfish (Black): c6d4
Model (White): f3d4 (confidence: 0.5453)
St

## Analyze a Position

In [11]:
# Analyze starting position
analyze_position()

# Analyze a specific FEN position
# Example: After 1.e4 e5 2.Nf3
# analyze_position("rnbqkbnr/pppp1ppp/8/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R b KQkq - 1 2")

Model's recommended move: e2e4
Confidence: 0.4121


Move.from_uci('e2e4')

## Configuration

### Stockfish Installation:

**Windows:**
1. Download from: https://stockfishchess.org/download/
2. Extract to a folder (e.g., `C:\stockfish`)
3. In the code, use: `stockfish_path=r'C:\stockfish\stockfish.exe'`

**macOS:**
```bash
brew install stockfish
```

**Linux:**
```bash
sudo apt-get install stockfish
```

### Adjust Stockfish Difficulty:
- Modify the `depth` parameter in `Stockfish(depth=10)` - higher = stronger
- Also supports `skill_level` parameter (0-20)